## Phase Detection and Signal Splitting

This script splits previously synchronized experimental data streams into two phases: **Reach and Grasp Phase** and **Lift and Hold Phase**. Segmentation is performed using averaged velocities of the hand movement. By detecting velocity thresholds and temporal patterns the start, lift onset and end of the experiment is found. Corresponding video files are also split into these phase-specific segments for visual control.

- **Parsing Utilities**: Extracts timestamps, camera indices, and match numbers from file and folder names.
- **Data Loading**: Loads synchronized time, kinematic, and myoelectric data from `.npy` files.
- **Smoothing**: Uses a low-pass filter to smooth velocity data.
- **Phase Detection**: Identifies phase transitions (start of reach, grasp, lift, and hold) using smoothed x- and y-velocity thresholds and temporal patterns.
- **Data Splitting**: Segments data into phases and saves them as `.npy` files.
- **Video Cropping**: Splits video recordings into phase-specific segments.
- **Debugging**: Creates annotated velocity plots with vertical dashed lines marking the detected phases.


In [2]:
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import cv2
from datetime import datetime
from scipy.signal import butter, filtfilt

# --------------------------------------------------------------------
# 1) Parsing Utilities for Camera Directory & Filenames
# --------------------------------------------------------------------
def parse_session_datetime(session_folder_name: str) -> datetime:
    """
    Parse folder name like 'session_YYYY-MM-DD_HH_MM_SS'
    => returns a datetime(YYYY, MM, DD, HH, MM, SS).
    """
    pattern = r"session_(\d{4}-\d{2}-\d{2})_(\d{2})_(\d{2})_(\d{2})"
    m = re.match(pattern, session_folder_name)
    if not m:
        raise ValueError(f"Cannot parse session folder: {session_folder_name}")
    date_str = m.group(1)  
    hh = int(m.group(2))
    mm = int(m.group(3))
    ss = int(m.group(4))

    dt_fmt = "%Y-%m-%d"
    base_date = datetime.strptime(date_str, dt_fmt)
    return datetime(
        year=base_date.year,
        month=base_date.month,
        day=base_date.day,
        hour=hh,
        minute=mm,
        second=ss
    )

def parse_recording_time(recording_folder_name: str):
    """
    Parse folder name like 'recording_HH_MM_SS.xxx'
    => returns (H, M, S_float).

    e.g. 'recording_10_42_26.955' => (10, 42, 26.955)
    """
    pattern = r"recording_(\d{2})_(\d{2})_(\d{2}\.\d+)$"
    m = re.match(pattern, recording_folder_name)
    if not m:
        raise ValueError(f"Cannot parse recording folder: {recording_folder_name}")
    hh = int(m.group(1))
    mm = int(m.group(2))
    ss_float = float(m.group(3))
    return (hh, mm, ss_float)

def parse_camera_index(video_filename: str) -> int:
    """
    For a file named 'annotated_video_camera_0.avi', returns int(0).
    If not found, returns -1.
    """
    pattern = r"annotated_video_camera_(\d+)\.avi"
    m = re.match(pattern, video_filename)
    if not m:
        return -1
    return int(m.group(1))

def parse_camera_timestamp_from_path(fullpath: str):
    """
    1) Find 'session_YYYY-MM-DD_HH_MM_SS' in path => parse date/time
    2) Find 'recording_HH_MM_SS.xxx' in path => parse time-of-day
    3) Combine them => final datetime
    4) Extract camera index from 'annotated_video_camera_<IDX>.avi'
    Returns (datetime, camera_index) or None if parse fails.
    """
    # Split the path into parts
    path_parts = fullpath.split(os.sep)

    # Parse session datetime
    session_folders = [p for p in path_parts if p.startswith("session_")]
    if not session_folders:
        return None
    session_folder_name = session_folders[-1]
    try:
        session_dt = parse_session_datetime(session_folder_name)
    except ValueError:
        return None

    # Parse recording time
    recording_folders = [p for p in path_parts if p.startswith("recording_")]
    if not recording_folders:
        return None
    recording_folder_name = recording_folders[-1]
    try:
        hh, mm, ss_float = parse_recording_time(recording_folder_name)
    except ValueError:
        return None

    # Combine session date with recording time
    combined_dt = session_dt.replace(
        hour=hh,
        minute=mm,
        second=int(ss_float),
        microsecond=int((ss_float - int(ss_float)) * 1e6)
    )

    # Parse camera index
    filename = os.path.basename(fullpath)
    camera_idx = parse_camera_index(filename)

    return (combined_dt, camera_idx)

def get_sorted_camera_files(camera_base_dir: str):
    """
    1) Glob all 'annotated_video_camera_*.avi' in camera_base_dir
    2) Parse session+recording datetime, camera index
    3) Sort by datetime, then camera index
    Returns a list of (filepath, datetime, camera_idx).
    """
    pattern = os.path.join(camera_base_dir, "**", "annotated_video_camera_*.avi")
    cam_files = glob.glob(pattern, recursive=True)
    results = []
    for cf in cam_files:
        parsed = parse_camera_timestamp_from_path(cf)
        if parsed is None:
            print(f"[WARN] Could not parse datetime from path: {cf}")
            continue
        dt, cam_idx = parsed
        results.append((cf, dt, cam_idx))

    # Sort by dt first, then camera_idx
    results.sort(key=lambda x: (x[1], x[2]))
    return results

# --------------------------------------------------------------------
# 2) Sorting Match Files
# --------------------------------------------------------------------
def parse_match_number(fname: str):
    """
    For 'match_01_time.npy', returns int(1).
    If no match, returns None.
    """
    pattern = r"match_(\d+)_time\.npy"
    m = re.match(pattern, fname)
    if not m:
        return None
    return int(m.group(1))

def get_sorted_match_files(sync_dir: str):
    """
    1) Glob all 'match_*_time.npy'
    2) Parse the numeric match index
    3) Sort ascending
    Returns list of (time_file_path, match_num).
    """
    time_files = glob.glob(os.path.join(sync_dir, "match_*_time.npy"))
    parsed_list = []
    for tf in time_files:
        base_name = os.path.basename(tf)
        match_num = parse_match_number(base_name)
        if match_num is not None:
            parsed_list.append((tf, match_num))

    parsed_list.sort(key=lambda x: x[1])  # sort by match_num
    return parsed_list

# --------------------------------------------------------------------
# 3) Load Data
# --------------------------------------------------------------------
def load_synchronized_data(sync_dir, match_prefix, hand_dims):
    """
    Loads time, kin, myo, otb from .npy files if they exist.
    Returns (time_data, kin_data, myo_data, otb_data) or None if missing files.
    """
    time_file = os.path.join(sync_dir, match_prefix + "_time.npy")
    kin_file  = os.path.join(sync_dir, match_prefix + "_kin.npy")
    myo_file  = os.path.join(sync_dir, match_prefix + "_myo.npy")
    otb_file  = os.path.join(sync_dir, match_prefix + "_otb.npy")

    if not all(os.path.exists(f) for f in [time_file, kin_file, myo_file, otb_file]):
        print(f"[WARN] Missing files for {match_prefix}.")
        return None

    time_data = np.load(time_file)
    kin_data  = np.load(kin_file)
    myo_data  = np.load(myo_file)
    otb_data  = np.load(otb_file)

    if len(time_data) < 5:
        print(f"[WARN] Not enough frames in {match_prefix}.")
        return None
    if kin_data.shape[1] < hand_dims:
        print(f"[WARN] kin_data has fewer than {hand_dims} columns.")
        return None

    return time_data, kin_data, myo_data, otb_data

# --------------------------------------------------------------------
# 4) Smoothing
# --------------------------------------------------------------------
def heavy_smooth(signal, dt, cutoff_hz=0.01, order=4):
    """
    Zero-phase Butterworth lowpass filter with filtfilt,
    so that we can smooth more without a phase shift.
    
    Parameters
    ----------
    signal : 1D array
        The raw data to filter
    dt : float
        The sampling interval (seconds between consecutive points)
    cutoff_hz : float
        The lowpass cutoff frequency in Hz (smaller => more smoothing)
    order : int
        The filter order (e.g., 4 => a 4th-order Butterworth)
    Returns
    -------
    filtered : 1D array
        The smoothed signal (same length as input)
    """
    fs = 1.0 / dt  # sampling rate in Hz
    nyquist = 0.5 * fs
    Wn = cutoff_hz / nyquist  # normalize cutoff to [0..1]
    b, a = butter(order, Wn, btype='low', analog=False)
    return filtfilt(b, a, signal)

# --------------------------------------------------------------------
# 5) Detect Task Boundaries and Phases
# --------------------------------------------------------------------
def detect_grasp_phases(
    time_data,
    kin_data,
    x_return_thresh=15.0,    # X "close to zero" threshold for iH
    x_end_thresh1=-80.0,     # X velocity threshold1 (< -50.0) for detecting i3_candidate
    x_end_thresh2=-10.0,     # X velocity threshold2 (<-10.0) for finding t3
    y_lift_thresh=10.0,       # Y-axis threshold for Phase 2 start
    y_zero_tol=5.0,          # Tolerance for zero crossing in Y velocity
    n_consec_frames=2,
    n_hand_landmarks=21
):
    """
    Detects grasp phases based on smoothed velocities. This version adds
    per-phase debug messages and partial detection results even if the
    overall detection fails.

    Returns a dictionary with keys:
      - 'failed': bool
      - 'i0','i1','i2','i3': int or None
      - 't0','t1','t2','t3': float or None
      - 'debug_i0','debug_i1','debug_i2','debug_i3': debug messages
      - 'x_vel_sm','y_vel_sm','vel_time': debugging velocity arrays
    """

    # ----------------------------------------------------------------
    # Initialize dictionary with placeholders
    # ----------------------------------------------------------------
    phase_result = {
        'failed': False,
        'i0': None, 'i1': None, 'i2': None, 'i3': None,
        't0': None, 't1': None, 't2': None, 't3': None,
        'debug_i0': "", 'debug_i1': "", 'debug_i2': "", 'debug_i3': "",
        'x_vel_sm': None, 'y_vel_sm': None, 'vel_time': None
    }

    # --------------------------------------------------------
    # 1) Compute time-step and check validity
    # --------------------------------------------------------
    dt = np.median(np.diff(time_data))
    if dt <= 0:
        phase_result['failed'] = True
        phase_result['debug_i0'] = "[WARN] dt <= 0 => invalid data."
        return phase_result

    # --------------------------------------------------------
    # 2) Compute & smooth velocities
    # --------------------------------------------------------
    # kin_data shape: (num_samples, n_hand_landmarks*3 + pose_dimensions)
    pos = kin_data[:, :n_hand_landmarks*3]  # Extract only hand landmark positions
    vel = (pos[1:] - pos[:-1]) / dt  # shape: (num_samples-1, n_hand_landmarks*3)

    x_inds = np.arange(0, 3*n_hand_landmarks, 3)
    y_inds = np.arange(1, 3*n_hand_landmarks, 3)

    raw_x_vel = np.mean(vel[:, x_inds], axis=1)
    raw_y_vel = np.mean(vel[:, y_inds], axis=1)

    cutoff_hz = 1.4  # can be tuned
    x_vel_sm = heavy_smooth(raw_x_vel, dt, cutoff_hz=cutoff_hz, order=4)
    y_vel_sm = heavy_smooth(raw_y_vel, dt, cutoff_hz=cutoff_hz, order=4)

    vel_time = 0.5 * (time_data[:-1] + time_data[1:])

    # Store in results for debugging/plotting
    phase_result['x_vel_sm'] = x_vel_sm
    phase_result['y_vel_sm'] = y_vel_sm
    phase_result['vel_time'] = vel_time

    # --------------------------------------------------------
    # 3) Try detecting i0
    # --------------------------------------------------------
    HIGH_VELOCITY_THRESHOLD = 50
    LOW_VELOCITY_THRESHOLD  = 10

    high_velocity_indices = np.where(x_vel_sm > HIGH_VELOCITY_THRESHOLD)[0]
    if high_velocity_indices.size == 0:
        phase_result['debug_i0'] = "[INFO] No velocity > 70 => i0 not found."
        phase_result['failed'] = True
        return phase_result

    first_high_idx = high_velocity_indices[0]
    i0_idx = None
    for i in range(first_high_idx, -1, -1):
        if x_vel_sm[i] < LOW_VELOCITY_THRESHOLD:
            i0_idx = i
            break

    if i0_idx is None:
        phase_result['debug_i0'] = "[INFO] Could not find a pre-high drop < 15 => i0 not found."
        phase_result['failed'] = True
        return phase_result
    else:
        t0 = vel_time[i0_idx]
        i0 = np.searchsorted(time_data, t0, side='left')
        phase_result['i0'] = i0
        phase_result['t0'] = t0
        phase_result['debug_i0'] = f"[OK] Found i0 at vel_time idx={i0_idx}, t0={t0:.3f}s"

    # --------------------------------------------------------
    # 4) Try detecting i3
    # --------------------------------------------------------
    i3_candidate_idx = None
    consec = 0

    # Scan backward to find the first valid region where x_vel_sm < x_end_thresh1
    for i in range(len(x_vel_sm) - 1, -1, -1):
        if x_vel_sm[i] < x_end_thresh1:  # e.g., -50.0 or -80.0
            consec += 1
        else:
            consec = 0
        if consec >= n_consec_frames:
            i3_candidate_idx = i - n_consec_frames + 1  # First valid region
            break

    if i3_candidate_idx is None or i3_candidate_idx <= i0_idx:
        # Fallback: set i3 to the end
        i3_idx = len(vel_time) - 1
        t3 = vel_time[i3_idx]
        i3 = len(time_data) - 1
        phase_result['debug_i3'] = "[INFO] No valid i3 found => fallback to end of data."
    else:
        # Refine i3 by moving left to find the first abs(x_vel_sm) < 10
        i3_idx_check = None
        for i in range(i3_candidate_idx, i0_idx - 1, -1):  # Go left
            if abs(x_vel_sm[i]) < 10.0:  # Absolute value check
                i3_idx_check = i
                break

        if i3_idx_check is None or i3_idx_check <= i0_idx:
            # Fallback to the end of the data if no valid crossing found
            i3_idx = len(vel_time) - 1
            phase_result['debug_i3'] = "[INFO] No crossing for i3 => fallback to end of data."
        else:
            # Use the refined index
            i3_idx = i3_idx_check

    # Set the final time and index for i3
    t3 = vel_time[i3_idx]
    i3 = np.searchsorted(time_data, t3, side='left')
    phase_result['i3'] = i3
    phase_result['t3'] = t3
    phase_result['debug_i3'] = f"[OK] Refined i3 at vel_time idx={i3_idx}, t3={t3:.3f}s"

    # --------------------------------------------------------
    # 5) Try detecting iH
    # --------------------------------------------------------
    iH_idx = None
    consec = 0
    for i in range(i0_idx, i3_idx + 1):
        if abs(x_vel_sm[i]) < x_return_thresh:
            consec += 1
        else:
            consec = 0
        if consec >= n_consec_frames:
            iH_idx = i - n_consec_frames + 1
            break

    if iH_idx is None:
        # fallback
        fallback_thresholds = [12.0, 15.0, 20.0]
        phase_result['debug_i1'] = f"[INFO] Did not find X near-zero (< {x_return_thresh}) => fallback."
        for alt_thresh in fallback_thresholds:
            consec = 0
            for i in range(i0_idx, i3_idx + 1):
                if abs(x_vel_sm[i]) < alt_thresh:
                    consec += 1
                else:
                    consec = 0
                if consec >= (n_consec_frames - 1):
                    iH_idx = i - (n_consec_frames - 1) + 1
                    phase_result['debug_i1'] += f" [OK] Found iH_idx with alt_thresh={alt_thresh}."
                    break
            if iH_idx is not None:
                break
        if iH_idx is None:
            phase_result['debug_i1'] += " [FAIL] Could not find iH."
            phase_result['failed'] = True
        else:
            tH = vel_time[iH_idx]
            phase_result['i1'] = np.searchsorted(time_data, tH, side='left')
            phase_result['t1'] = tH
    else:
        tH = vel_time[iH_idx]
        phase_result['i1'] = np.searchsorted(time_data, tH, side='left')
        phase_result['t1'] = tH
        phase_result['debug_i1'] = f"[OK] Found iH at vel_time idx={iH_idx}, tH={tH:.3f}s"

    # If iH was found but we want to add 1s shift
    if phase_result['i1'] is not None and not phase_result['failed']:
        iH_plus_idx = np.searchsorted(vel_time, tH + 2.0, side='left')
        if iH_plus_idx >= len(vel_time):
            iH_plus_idx = len(vel_time) - 1
        tH_plus = vel_time[iH_plus_idx]
        phase_result['i1'] = np.searchsorted(time_data, tH_plus, side='left')
        phase_result['t1'] = tH_plus
        phase_result['debug_i1'] += f" [SHIFT] iH => iH+1s at vel_time idx={iH_plus_idx}, tH={tH_plus:.3f}s"

    # --------------------------------------------------------
    # 6) Try detecting i2
    # --------------------------------------------------------
    # Only proceed if iH was found
    if phase_result['i1'] is None or phase_result['failed']:
        # We already failed, but keep partial results
        return phase_result

    iH_idx_updated = np.searchsorted(vel_time, phase_result['t1'], side='left')
    i2_dip_start = None
    consec = 0
    for i in range(iH_idx_updated, i3_idx + 1):
        if y_vel_sm[i] < -y_lift_thresh:
            consec += 1
        else:
            consec = 0
        if consec >= n_consec_frames:
            i2_dip_start = i - n_consec_frames + 1
            break

    if i2_dip_start is None:
        phase_result['debug_i2'] = f"[INFO] No Y-lift crossing found (< -{y_lift_thresh})."
        phase_result['failed'] = True
        return phase_result

    # Find the near-zero crossing in y velocity
    i2_idx = None
    for j in range(i2_dip_start, iH_idx_updated, -1):
        if abs(y_vel_sm[j]) < y_zero_tol:
            i2_idx = j
            break
    if i2_idx is None:
        i2_idx = i2_dip_start

    t2 = vel_time[i2_idx]
    i2 = np.searchsorted(time_data, t2, side='left')
    phase_result['i2'] = i2
    phase_result['t2'] = t2
    phase_result['debug_i2'] = f"[OK] Found i2 at vel_time idx={i2_idx}, t2={t2:.3f}s"

    # Final check
    if not (phase_result['i1'] <= i2 < i3):
        phase_result['debug_i2'] += " [FAIL] i2 not in [iH, i3)."
        phase_result['failed'] = True

    return phase_result

# --------------------------------------------------------------------
# 6) Splitting & Cropping - Single Camera Version
# --------------------------------------------------------------------
def split_save_and_crop_video_single_camera(
    match_prefix,
    time_data, kin_data, myo_data, otb_data,
    phase_dict,
    out_dir,
    camera_file,
    camera_out_dir
):
    """
    Saves Phase1 & Phase2 data as .npy,
    then crops the single camera_file into Phase1.avi and Phase2.avi.

    We assume phase_dict has keys: 'i0', 'i2', 'i3'.
    Phase1 = [i0, i2), Phase2 = [i2, i3).
    """

    i0 = phase_dict['i0']
    i2 = phase_dict['i2']
    i3 = phase_dict['i3']

    # PHASE 1 => [i0, i2)
    t_phase1  = time_data[i0:i2]
    kin_p1    = kin_data[i0:i2]
    myo_p1    = myo_data[i0:i2]
    otb_p1    = otb_data[i0:i2]

    # PHASE 2 => [i2, i3)
    t_phase2  = time_data[i2:i3]
    kin_p2    = kin_data[i2:i3]
    myo_p2    = myo_data[i2:i3]
    otb_p2    = otb_data[i2:i3]

    p1_prefix = match_prefix + "_phase1"
    p2_prefix = match_prefix + "_phase2"

    # Save each phase
    np.save(os.path.join(out_dir, p1_prefix + "_time.npy"), t_phase1)
    np.save(os.path.join(out_dir, p1_prefix + "_kin.npy"),  kin_p1)
    np.save(os.path.join(out_dir, p1_prefix + "_myo.npy"),  myo_p1)
    np.save(os.path.join(out_dir, p1_prefix + "_otb.npy"),  otb_p1)

    np.save(os.path.join(out_dir, p2_prefix + "_time.npy"), t_phase2)
    np.save(os.path.join(out_dir, p2_prefix + "_kin.npy"),  kin_p2)
    np.save(os.path.join(out_dir, p2_prefix + "_myo.npy"),  myo_p2)
    np.save(os.path.join(out_dir, p2_prefix + "_otb.npy"),  otb_p2)

    print(f"[SAVED] {match_prefix} => Phase1: {len(t_phase1)} frames, Phase2: {len(t_phase2)} frames.")

    # Video cropping
    def time_to_frame(tsec, fps, total_frames):
        frame_idx = int(round(tsec * fps))
        return max(0, min(frame_idx, total_frames - 1))

    # Convert i0, i2, i3 -> times
    t0_ = time_data[i0]
    t2_ = time_data[i2]
    t3_ = time_data[i3] if i3 < len(time_data) else time_data[-1]

    cam_name = os.path.basename(camera_file)
    cam_sub  = os.path.join(camera_out_dir, cam_name.replace(".avi",""))
    os.makedirs(cam_sub, exist_ok=True)

    cap = cv2.VideoCapture(camera_file)
    if not cap.isOpened():
        print(f"[WARN] Could not open {camera_file}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_f = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # PHASE 1 => [t0_, t2_)
    p1_file = os.path.join(cam_sub, f"{match_prefix}_phase1_{cam_name}")
    s1 = time_to_frame(t0_, fps, total_f)
    e1 = time_to_frame(t2_, fps, total_f)
    if s1 < e1:
        out1 = cv2.VideoWriter(p1_file, fourcc, fps, (w,h))
        cap.set(cv2.CAP_PROP_POS_FRAMES, s1)
        idx = s1
        while idx < e1:
            ret, frame = cap.read()
            if not ret:
                break
            out1.write(frame)
            idx += 1
        out1.release()

    # PHASE 2 => [t2_, t3_)
    p2_file = os.path.join(cam_sub, f"{match_prefix}_phase2_{cam_name}")
    s2 = time_to_frame(t2_, fps, total_f)
    e2 = time_to_frame(t3_, fps, total_f)
    if s2 < e2:
        out2 = cv2.VideoWriter(p2_file, fourcc, fps, (w,h))
        cap.set(cv2.CAP_PROP_POS_FRAMES, s2)
        idx = s2
        while idx < e2:
            ret, frame = cap.read()
            if not ret:
                break
            out2.write(frame)
            idx += 1
        out2.release()

    cap.release()

def get_cropped_camera_files(cropped_videos_dir: str):
    pattern = os.path.join(cropped_videos_dir, "match_*_cropped_camera_*.avi")
    cam_files = glob.glob(pattern)
    results = []
    for cf in cam_files:
        base_name = os.path.basename(cf)
        m = re.match(r"match_(\d+)_cropped_camera_(\d+)\.avi", base_name)
        if not m:
            continue
        match_num = int(m.group(1))
        cam_idx   = int(m.group(2))
        results.append((cf, match_num, cam_idx))
    results.sort(key=lambda x: (x[1], x[2]))  # Sort by match_num, then camera_idx
    return results

# --------------------------------------------------------------------
# 7) Debug Plot
# --------------------------------------------------------------------
def plot_debug_velocities(
    match_prefix,
    time_data,
    phase_info,
    out_dir=None,
    show_plot=False
):
    """
    Always plots velocities. If some of i0, iH, i2, i3 were found,
    draw vertical lines for them. If a phase was not found, skip it.
    Also annotate the debug messages in the console.
    """
    x_vel = phase_info['x_vel_sm']
    y_vel = phase_info['y_vel_sm']
    vel_time = phase_info['vel_time']

    # Print out debug info for each phase index
    print("\n[DEBUG] " + match_prefix)
    print("   i0 =>", phase_info['debug_i0'])
    print("   iH =>", phase_info['debug_i1'])
    print("   i2 =>", phase_info['debug_i2'])
    print("   i3 =>", phase_info['debug_i3'])

    if x_vel is None or y_vel is None or vel_time is None:
        print("[DEBUG] No velocity data => skipping plot.")
        return

    fig_title = f"Velocity Debug: {match_prefix}"
    if phase_info['failed']:
        fig_title += " (DETECTION FAILED)"

    fig, axs = plt.subplots(2, 1, figsize=(10,6), sharex=True)
    fig.suptitle(fig_title)

    # Plot X velocity
    axs[0].plot(vel_time, x_vel, label="Avg X Vel (smoothed)")
    axs[0].set_ylabel("X Velocity [units/s]")
    axs[0].legend()

    # Plot Y velocity
    axs[1].plot(vel_time, y_vel, label="Avg Y Vel (smoothed)", color='tab:orange')
    axs[1].set_ylabel("Y Velocity [units/s]")
    axs[1].set_xlabel("Time (s)")
    axs[1].legend()

    # Always attempt to plot lines if they exist
    i0, i1, i2, i3 = phase_info['i0'], phase_info['i1'], phase_info['i2'], phase_info['i3']
    t0, t1, t2, t3 = phase_info['t0'], phase_info['t1'], phase_info['t2'], phase_info['t3']

    # Each of these might be None if detection failed partially:
    def vline_if_found(ax, t, label, color):
        if t is not None:
            ax.axvline(t, color=color, linestyle='--', label=label)

    # top subplot => i0 & i3
    vline_if_found(axs[0], t0, 'i0', 'green')
    vline_if_found(axs[0], t3, 'i3', 'red')
    axs[0].legend()

    # bottom subplot => i0, iH, i2, i3
    vline_if_found(axs[1], t0, 'i0', 'green')
    vline_if_found(axs[1], t1, 'iH', 'blue')
    vline_if_found(axs[1], t2, 'i2', 'magenta')
    vline_if_found(axs[1], t3, 'i3', 'red')
    axs[1].legend()

    plt.tight_layout()

    # Save or show
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)
        suffix = "_debug_vel.png"
        if phase_info['failed']:
            suffix = "_debug_vel_FAILED.png"
        fpath = os.path.join(out_dir, f"{match_prefix}{suffix}")
        plt.savefig(fpath, dpi=150)
        print(f"[SAVED] Debug plot => {fpath}")

    if show_plot:
        plt.show()
    plt.close(fig)

# --------------------------------------------------------------------
# 8) Main
# --------------------------------------------------------------------
def main():
    BASE_DIR = r"C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data"
    PARTICIPANTS = [2,3,4,5,6,7,8]

    SYNC_FOLDER   = "Synchronized Data"
    SPLIT_FOLDER  = "Synchronized Data split in Phases"
    CROPPED_VIDEOS_SUBFOLDER = "Cropped Videos"

    n_hand_landmarks = 21
    hand_dims = n_hand_landmarks * 3

    # Phase detection parameters
    X_RETURN_THRESH = 15.0
    X_END_THRESH1   = -50.0
    X_END_THRESH2   = 10.0
    Y_LIFT_THRESH   = 10.0
    N_CONSEC        = 2
    Y_ZERO_TOL      = 5.0  # Removed trailing comma to make it a float

    for p in PARTICIPANTS:
        participant_str = f"P({p})"
        participant_dir = os.path.join(BASE_DIR, participant_str)
        sync_dir = os.path.join(participant_dir, SYNC_FOLDER)
        if not os.path.isdir(sync_dir):
            print(f"[WARN] No Synchronized Data folder for {participant_str}.")
            continue

        split_dir = os.path.join(participant_dir, SPLIT_FOLDER)
        os.makedirs(split_dir, exist_ok=True)

        # We look for cropped videos in:
        cropped_videos_dir = os.path.join(sync_dir, CROPPED_VIDEOS_SUBFOLDER)
        camera_out_dir     = os.path.join(split_dir, "Camera")
        os.makedirs(camera_out_dir, exist_ok=True)

        # 1) Sorted match files
        sorted_matches = get_sorted_match_files(sync_dir)
        if not sorted_matches:
            print(f"[INFO] No matched files found for {participant_str}.")
            continue

        # 2) Sorted camera files
        sorted_cam = get_cropped_camera_files(cropped_videos_dir)
        if not sorted_cam:
            print(f"[INFO] No cropped camera videos found in {cropped_videos_dir}")
            continue

        # Pair up by index or match_num (here, we just do index-based pairing)
        N = min(len(sorted_matches), len(sorted_cam))
        if N < len(sorted_matches):
            print(f"[WARN] Fewer camera files ({len(sorted_cam)}) than match files ({len(sorted_matches)})")

        # Debug-plot folder
        debug_plot_dir = os.path.join(split_dir, "Plots_Velocity_Debug")
        os.makedirs(debug_plot_dir, exist_ok=True)

        for i in range(N):
            match_time_file, match_num = sorted_matches[i]
            cam_file, cam_match_num, cam_idx = sorted_cam[i]

            # e.g. match_01_time.npy => match_prefix='match_01'
            match_prefix = os.path.basename(match_time_file).replace("_time.npy","")

            data_tuple = load_synchronized_data(sync_dir, match_prefix, hand_dims)
            if data_tuple is None:
                print(f"[WARN] Data missing for {match_prefix}, skipping.")
                continue

            time_data, kin_data, myo_data, otb_data = data_tuple

            # Detect phases
            phase_info = detect_grasp_phases(
                time_data, 
                kin_data,
                x_return_thresh = X_RETURN_THRESH,
                x_end_thresh1   = X_END_THRESH1,
                x_end_thresh2   = X_END_THRESH2,
                y_lift_thresh   = Y_LIFT_THRESH,
                y_zero_tol      = Y_ZERO_TOL,  # Now correctly a float
                n_consec_frames = N_CONSEC,
                n_hand_landmarks= n_hand_landmarks
            )

            # Always debug-plot
            plot_debug_velocities(
                match_prefix,
                time_data,
                phase_info,
                out_dir=debug_plot_dir,
                show_plot=False
            )

            # Only proceed with cropping if detection didn't fail
            if not phase_info['failed']:
                split_save_and_crop_video_single_camera(
                    match_prefix,
                    time_data, kin_data, myo_data, otb_data,
                    phase_info,
                    out_dir       = split_dir,
                    camera_file   = cam_file,
                    camera_out_dir= camera_out_dir
                )
            else:
                print(f"[INFO] Could not detect phases for {match_prefix}, skipping cropping.")

if __name__ == "__main__":
    main()



[DEBUG] match_01
   i0 => [OK] Found i0 at vel_time idx=3712, t0=1.856s
   iH => [OK] Found iH at vel_time idx=3712, tH=1.856s [SHIFT] iH => iH+1s at vel_time idx=7712, tH=3.856s
   i2 => [OK] Found i2 at vel_time idx=10387, t2=5.194s
   i3 => [OK] Refined i3 at vel_time idx=27442, t3=13.721s
[SAVED] Debug plot => C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(2)\Synchronized Data split in Phases\Plots_Velocity_Debug\match_01_debug_vel.png
[SAVED] match_01 => Phase1: 6675 frames, Phase2: 17055 frames.

[DEBUG] match_02
   i0 => [OK] Found i0 at vel_time idx=4609, t0=2.305s
   iH => [OK] Found iH at vel_time idx=4609, tH=2.305s [SHIFT] iH => iH+1s at vel_time idx=8609, tH=4.305s
   i2 => [OK] Found i2 at vel_time idx=11963, t2=5.982s
   i3 => [OK] Refined i3 at vel_time idx=28579, t3=14.290s
[SAVED] Debug plot => C:\Users\schmi\Documents\Studium\TUM\5. Semester\Masterthesis\Experimental Data\P(2)\Synchronized Data split in Phases\Plots_Velocity_Debug\